# Cypher Keywords

## `CREATE`

`CREATE` -> Used to create nodes in a graph.
* **Creating Nodes**
    * **WARNING:** Although this section is focused on using `CREATE`, it does have the ability to generate duplicate nodes. For example, `CREATE (n:Person {id: 1}) RETURN n;` will generate a new 'Person' node with property: '{id: 1}' as many times as the query is executed. However, swapping `CREATE` for `MERGE` will prevent duplicates from appearing in the graph.
        * It is better practice to use `MERGE` over `CREATE`
    * Create a node with single labels by using: `MERGE (n:Person) RETURN n`. Specify multiple labels like `MERGE (n:Person:Animal:Professional) RETURN n`
    * Create a node with properties inside curly brackets like so: `MERGE (n:Person {name: "Henry", age: 17}) RETURN n`

* **Creating Relationships**
    * This is best illustrated through an example. Let's say Sally goes to the store to buy some pickles. To represent this graphically, first create two nodes for entities Sally (person) and pickles (food). The action or relationship between the two is the act of purchasing. So, in cypher we would write:
        > `CREATE (p:Person {name: "Sally"}), (f:Food {item: "Pickles"}), (p)-[:PURCHASES]->(f) RETURN p, f;`

        > `MERGE (p:Person {name: "Sally"})`<br>
          `MERGE (f:Food {item: "Pickles"})`<br>
          `MERGE (p)-[:PURCHASES]->(f)`<br>
          `RETURN p, f;`<br>

`RETURN` -> Instructed the graph to send back data from the graph based on what follows the 'RETURN' keyword.
* `RETURN *` will return all variables and properties from the query.

In [1]:
from neo4j import GraphDatabase, Record, ResultSummary, EagerResult
import os 
import socket
from dotenv import load_dotenv 
load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

**NOTE:**

When working from WSL2 and Neo4j Desktop is installed on the Windows side, you have to set up port forwarding. To do this, open a Powershell administrator window and run the following:
1. Run `ipconig`
* From this point forward assume you have a Windows ip address of: '123.456.78.900'
2. Launch the Neo4j database you want to query and take note of the port number (at the time of righting this, the default bolt port is '7687').
3. Run `netsh interface portproxy set v4tov4 listenport=7687 listenaddress=123.456.78.900 connectport=7687 connectaddress=127.0.0.1`
4. To verify, run `netsh interface portproxy show v4tov4`
5. To disable the port forwarding, run `netsh interface portproxy delete v4tov4 listenport=1234 listenaddress=123.456.78.900`

If working from a windows environment where Neo4j Desktop is installed, the default 'localhost' URI should be sufficient.

In [2]:
def annotate_results(result: EagerResult, verbose: bool = False) -> list[dict]:
    """ 
    A helper function to optionally print out some helpful metadata around the query and return
    a parsed list from the EagerResult.records object.mro

    Args:
        result -> The EagerResult object returned by Neo4j
        verbose -> Optional parameter to print out informative query metadata or not

    Returns:
        A list of records as dictionaries
    """
    start = result.summary.result_available_after
    finish = start + result.summary.result_consumed_after
    print(f"Started streaming {len(result.records)} records after {start} ms and completed after {finish} ms.")
    
    if verbose:
        print(f"Query executed (database '{result.summary.database}'): {result.summary.query}")

    return [record.data() for record in result.records]

What the heck is an 'EagerResult'?

https://neo4j.com/docs/api/python-driver/current/api.html#neo4j.EagerResult

In [3]:
# Let's see how many nodes are in our graph
result: EagerResult[list[Record], ResultSummary, list[str]] = driver.execute_query(
    """ 
    MATCH (n) RETURN n;
    """,
    database_="dev"
)

lst = annotate_results(result, True)
lst

Started streaming 0 records after 3 ms and completed after 4 ms.
Query executed (database 'dev'):  
    MATCH (n) RETURN n;
    


[]

**NOTE:**

To keep things clean, I'll only annotate the 'result' variable once. Clearly the response returned from Neo4j is a pretty complex object that holds a ton of useful information. I would encourage you to explore their documentation and modify the `annotate_results` function to provide you with information that is the most helpful to you!

In [4]:
# Let's create a person in our graph named Sally
result = driver.execute_query(
    """ 
    CREATE (p:Person {name: "Sally"})
    RETURN p
    """,
    database_="dev"
)

lst = annotate_results(result, True)

# Let's take a look at how our record is represented in python
lst

Started streaming 1 records after 46 ms and completed after 48 ms.
Query executed (database 'dev'):  
    CREATE (p:Person {name: "Sally"})
    RETURN p
    


[{'p': {'name': 'Sally'}}]

In [5]:
# Let's delete all records in Neo4j
result = driver.execute_query(
    """ 
    MATCH (n) DETACH DELETE n;
    """,
    database_="dev"
)

lst = annotate_results(result, True)

Started streaming 0 records after 0 ms and completed after 0 ms.
Query executed (database 'dev'):  
    MATCH (n) DETACH DELETE n;
    


In [6]:
# Let's create a relationship with one direction to represent the scenario "Sally goes to the store and purchases pickles"
result = driver.execute_query(
    """ 
    CREATE (p:Person {name: "Sally"}), (f:Food {item: "Pickles"}), (p)-[r:PURCHASES]->(f) RETURN *;
    """,
    database_="dev"
)

lst = annotate_results(result, True)

Started streaming 1 records after 5 ms and completed after 7 ms.
Query executed (database 'dev'):  
    CREATE (p:Person {name: "Sally"}), (f:Food {item: "Pickles"}), (p)-[r:PURCHASES]->(f) RETURN *;
    
